In [10]:
import selenium
from selenium import webdriver
import time
import requests
import os
from PIL import Image
import io
import hashlib

In [11]:
# This is the path I use
# DRIVER_PATH = '/Users/anand/Desktop/chromedriver'
# Put the path for your ChromeDriver here
query = 'peregrine falcon flying'
DRIVER_PATH = '/Applications/chromedriver'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)
wd.get('https://google.com') 
search_box = wd.find_element_by_css_selector('input.gLFyf') #input box selector
search_box.send_keys(query)
# wd.quit()

In [12]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(10)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

def persist_image(folder_path:str,file_name:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        folder_path = os.path.join(folder_path,file_name)
        if os.path.exists(folder_path):
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.png')
        else:
            os.mkdir(folder_path)
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.png')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [13]:
links = fetch_image_urls(query, 150, wd, 1)

Found: 100 search results. Extracting links from 0:100
Found: 151 image links, done!


In [14]:
for link in links:
    persist_image('per_imgs/new_dataset/pos',query,link)

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQCGsmtfNgW3tO-zeIJK-zUXjI-6t3HA7HAmw&usqp=CAU - as per_imgs/new_dataset/pos/peregrine falcon flying/d02ac14e44.png
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQckC-Q5EqAJSTGtxhsSWBSj5IwRa7W_lfzLw&usqp=CAU - as per_imgs/new_dataset/pos/peregrine falcon flying/1c3343f79b.png
SUCCESS - saved https://www.birdsoutsidemywindow.org/wp-content/uploads/2009/06/pefa_ledge_walking_j9m8006_rsz_kims.jpg - as per_imgs/new_dataset/pos/peregrine falcon flying/0292fb82b6.png
SUCCESS - saved https://www.pge.com/pge_global/local/images/data/en-us/in-your-community/local-environment/peregrine-falcons/flying-falcon.jpg - as per_imgs/new_dataset/pos/peregrine falcon flying/3064e228ce.png
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRA653giQnrYwduAhOKAV0vVZBQfsrwtDfNmw&usqp=CAU - as per_imgs/new_dataset/pos/peregrine falcon flying/b61e586148.png
ERROR - Could not save https://news.berkeley

In [21]:
!pwd

/Users/emma-clairemccarthy/Desktop/Metis/projects/falcon_alert
